In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from tqdm import tqdm

#rest of the preprocessing is using the script provided in the TBIP repo - setup/senate_speeches_to_bag_of_words.py 
import os

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
raw_documents = list(map(lambda x:x.rstrip(), open("clean/raw_documents.txt").readlines()))
print(len(raw_documents))

84063


In [16]:
raw_data = pd.read_csv('finalized_tbip_speech_set_raw_original_data_floor_speeches_house.csv')
speeches_og = list(raw_data['Text'])
print(len(speeches_og))

84063


In [5]:
raw_docs_proc_rem = list(map(lambda x:x.rstrip(), open("raw_documents_without_procedural.txt").readlines()))
print(len(raw_docs_proc_rem))

59428


In [17]:
speeches_og = [document.replace("\n", ' ').replace("\r", ' ').rstrip()
                 for i, document in enumerate(speeches_og)]

In [18]:
speeches_og == raw_documents

True

In [20]:
retained_speech_inds = []
for i, s in tqdm(enumerate(speeches_og)):
    if s in raw_docs_proc_rem:
        retained_speech_inds.append(i)
print(len(retained_speech_inds))

84063it [01:09, 1205.30it/s]

59428


In [21]:
assert len(raw_docs_proc_rem) == len(retained_speech_inds)

In [22]:
raw_data = raw_data.iloc[retained_speech_inds]
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59428 entries, 0 to 84062
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Speaker_Bioguide_ID  59428 non-null  object
 1   Speaker_Name         59428 non-null  object
 2   Text                 59428 non-null  object
 3   Date                 59428 non-null  object
 4   Legislative Body     59428 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB
None


In [23]:
#remove speakers if they gave less than 25 speeches - TBIP paper removed senators with less than 24 speeches.
speakers_to_remove_based_on_num_speeches = set()
speakers = set(raw_data['Speaker_Bioguide_ID'])
thresh = 25
for s in speakers:
    n_s = len(raw_data[raw_data['Speaker_Bioguide_ID']==s])
    if n_s < thresh:
        speakers_to_remove_based_on_num_speeches.add(s)
print(len(speakers_to_remove_based_on_num_speeches))

23


In [24]:
raw_data = raw_data[~raw_data['Speaker_Bioguide_ID'].isin(speakers_to_remove_based_on_num_speeches)]
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58941 entries, 0 to 84062
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Speaker_Bioguide_ID  58941 non-null  object
 1   Speaker_Name         58941 non-null  object
 2   Text                 58941 non-null  object
 3   Date                 58941 non-null  object
 4   Legislative Body     58941 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB
None


In [25]:
stopwords = open('stopwords.txt').readlines()
stopwords = list(map(lambda x:x.rstrip(), stopwords))
print(len(stopwords))

1456


In [29]:
stopwords_proc = open('procedural_stopwords.txt').readlines()
stopwords_proc = list(map(lambda x:x.rstrip(), stopwords_proc))
print(len(stopwords_proc))

110


In [30]:
stopwords_proc[-5:]

['mr_speaker_i', 'and_include_extraneous', 'i_ask', 'their_remarks', 'members']

In [31]:
stopwords_proc[:5]

['remarks', '5', 'revise_and_extend', 'to_revise_and', 'mr']

In [32]:
stopwords_proc = list(map(lambda x:' '.join(x.split('_')), stopwords_proc))
stopwords_proc[:5]

['remarks', '5', 'revise and extend', 'to revise and', 'mr']

In [33]:
stopwords = list(set(stopwords).union(set(stopwords_proc)))
print(len(stopwords))

1527


In [34]:
speakers = list(raw_data['Speaker_Bioguide_ID'])
print(len(speakers))
speeches = list(raw_data['Text'])
print(len(speeches))

58941
58941


In [35]:
speaker_to_speaker_id = dict(
    [(y, x) for x, y in enumerate(sorted(set(speakers)))])
author_indices = np.array(
    [speaker_to_speaker_id[s] for s in speakers])
print(len(author_indices))
author_map = np.array(list(speaker_to_speaker_id.keys()))
print(len(author_map))

58941
490


In [36]:
print(len(speeches))

58941


In [37]:
count_vectorizer = CountVectorizer(min_df=0.001,
                                   max_df=0.75, 
                                   stop_words=stopwords, 
                                   ngram_range=(1, 3),
                                   token_pattern="[a-zA-Z]+")
# Learn initial document term matrix. This is only initial because we use it to
# identify words to exclude based on author counts.
counts = count_vectorizer.fit_transform(speeches)
vocabulary = np.array(
    [k for (k, v) in sorted(count_vectorizer.vocabulary_.items(), 
                            key=lambda kv: kv[1])])


/workspace/.conda/envs/tbip/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['allard', 'andr', 'balart', 'barrag', 'col', 'colon', 'cortez', 'garc', 'gonz', 'guti', 'halleran', 'jes', 'jos', 'jr', 'legislative', 'lehtinen', 'lez', 'luj', 'mucarsel', 'nchez', 'ocasio', 'powell', 'ra', 'rdenas', 'ros', 'rourke', 'roybal', 'rrez', 'shea', 'sr', 'vel', 'wm', 'zquez'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [38]:
print(counts.shape)
print(len(vocabulary))

(58941, 14090)
14090


In [39]:
author_to_inds = {}
for a in tqdm(list(author_map)):
    inds = []
    author_ind = speaker_to_speaker_id[a]
    for i, ind in enumerate(list(author_indices)):
        if ind==author_ind:
            inds.append(i)
    author_to_inds[a] = inds

100%|██████████| 490/490 [00:09<00:00, 50.92it/s]


In [40]:
def get_per_author_counts(counts, author_to_inds):
    list_of_arrays = []
    for a in author_to_inds:
        inds = author_to_inds[a]
        list_of_arrays.append(np.array(np.sum(counts[inds], 0)))#.reshape((1, counts.shape[1])))
    return np.concatenate(list_of_arrays, 0)

In [41]:
# Remove phrases spoken by less than 50 representatives
min_authors_per_word = 50
counts_per_author = get_per_author_counts(counts, author_to_inds)
print(counts_per_author.shape)

(490, 14090)


In [42]:
acceptable_words = []
for i in range(len(vocabulary)):
    if np.count_nonzero(counts_per_author[:, i]) >= min_authors_per_word:
        acceptable_words.append(i)
print(len(acceptable_words))

11433


In [43]:
count_vectorizer = CountVectorizer(ngram_range=(1, 3),
                                   vocabulary=vocabulary[acceptable_words])
counts = count_vectorizer.fit_transform(speeches)
vocabulary = np.array(
    [k for (k, v) in sorted(count_vectorizer.vocabulary_.items(), 
                            key=lambda kv: kv[1])])
print(counts.shape)
print(len(vocabulary))

(58941, 11433)
11433


In [44]:
# `n_gram_to_unigram` takes as key an index to an n-gram in the vocabulary
# and its value is a list of the vocabulary indices of the corresponding 
# unigrams.
n_gram_indices = np.where(
  np.array([len(word.split(' ')) for word in vocabulary]) > 1)[0]
n_gram_to_unigrams = {}
for n_gram_index in n_gram_indices:
    matching_unigrams = []
    for unigram in vocabulary[n_gram_index].split(' '):
        if unigram in vocabulary:
            matching_unigrams.append(np.where(vocabulary == unigram)[0][0])
    n_gram_to_unigrams[n_gram_index] = matching_unigrams

# `n_grams_to_bigrams` now breaks apart trigrams and higher to find bigrams 
# as subsets of these words.
n_grams_to_bigrams = {}
for n_gram_index in n_gram_indices:
    split_n_gram = vocabulary[n_gram_index].split(' ')
    n_gram_length = len(split_n_gram) 
    if n_gram_length > 2:
        bigram_matches = []
        for i in range(0, n_gram_length - 1):
            bigram = " ".join(split_n_gram[i:(i + 2)])
            if bigram in vocabulary:
                bigram_matches.append(np.where(vocabulary == bigram)[0][0])
        n_grams_to_bigrams[n_gram_index] = bigram_matches

In [45]:
# Go through counts, and remove a unigram each time a bigram superset 
# appears. Also remove a bigram each time a trigram superset appears.
# Note this isn't perfect: if bigrams overlap (e.g. "global health care" 
# contains "global health" and "health care"), we count them both. This
# may introduce a problem where we subract a unigram count twice, so we also
# ensure non-negativity.
#counts_dense = counts.toarray()
for i in tqdm(range(counts.shape[0])):
    n_grams_in_doc = np.where(counts[i, n_gram_indices].toarray() > 0)[0]
    sub_n_grams = n_gram_indices[n_grams_in_doc]
    for n_gram in sub_n_grams:
        counts[i, n_gram_to_unigrams[n_gram]] = sparse.csr_matrix(counts[i, n_gram_to_unigrams[n_gram]].toarray() - counts[i, n_gram])
        if n_gram in n_grams_to_bigrams:
            counts[i, n_grams_to_bigrams[n_gram]] = sparse.csr_matrix(counts[i, n_grams_to_bigrams[n_gram]].toarray() - counts[i, n_gram])
counts[counts < 0] = 0

100%|██████████| 58941/58941 [03:09<00:00, 311.55it/s]


In [46]:
print(counts.shape)

(58941, 11433)


In [47]:
# Remove speeches with no words.
existing_speeches = []#np.where(np.sum(counts_dense, axis=1) > 0)[0]
for i in tqdm(range(counts.shape[0])):
    if counts[i].sum() > 0:
        existing_speeches.append(i)
counts = counts[existing_speeches]
print(counts.shape)
author_indices = author_indices[existing_speeches]
print(author_indices.shape)

100%|██████████| 58941/58941 [00:09<00:00, 6066.27it/s]

(58932, 11433)
(58932,)


In [48]:
# Save data.

# `counts.npz` is a [num_documents, num_words] sparse matrix containing the
# word counts for each document.
sparse.save_npz("clean_removing_procedural/counts.npz",
                counts.astype(np.float32))

# `author_indices.npy` is a [num_documents] vector where each entry is an
# integer indicating the author of the corresponding document.
np.save("clean_removing_procedural/author_indices.npy", author_indices)

# `vocabulary.txt` is a [num_words] vector where each entry is a string
# denoting the corresponding word in the vocabulary.
np.savetxt("clean_removing_procedural/vocabulary.txt", vocabulary, fmt="%s")

# `author_map.txt` is a [num_authors] vector of strings providing the bioguide ID of
# each author in the corpus.
np.savetxt("clean_removing_procedural/author_map.txt", author_map, fmt="%s")

In [49]:
# `raw_documents.txt` contains all the documents we ended up using.
raw_documents = [document.replace("\n", ' ').replace("\r", ' ') 
                 for i, document in enumerate(speeches) if i in existing_speeches]

In [50]:
print(len(raw_documents))

58932


In [51]:
f = open('clean_removing_procedural/raw_documents.txt', 'w')
for i, doc in enumerate(raw_documents):
    f.write(doc)
    if i < len(raw_documents) - 1:
        f.write('\n')
f.close()

In [52]:
raw_data = raw_data.iloc[existing_speeches]
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58932 entries, 0 to 84062
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Speaker_Bioguide_ID  58932 non-null  object
 1   Speaker_Name         58932 non-null  object
 2   Text                 58932 non-null  object
 3   Date                 58932 non-null  object
 4   Legislative Body     58932 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB
None


In [53]:
raw_data.to_csv('finalized_tbip_speech_set_raw_original_data_floor_speeches_house_after_removing_procedural_speeches.csv',
                index=False)